<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Web/blob/main/202-Llamaindex-Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai==0.28

In [ ]:
!pip show openai | grep Version

In [ ]:
!pip install llama-index

### Preparation

In [ ]:
import os
import openai

os.environ["OPENAI_API_KEY"] = "sk-"
openai.api_key = os.environ["OPENAI_API_KEY"]

import nest_asyncio

nest_asyncio.apply()

### Ingest Data - Uber 공시 자료

In [ ]:
# NOTE: the code examples assume you're operating within a Jupyter notebook.
# download files
!mkdir data
!wget "https://www.dropbox.com/s/948jr9cfs7fgj99/UBER.zip?dl=1" -O data/UBER.zip
!unzip data/UBER.zip -d data

### HTML 파일을 서식 있는 텍스트로 구문 분석하기 위해 구조화되지 않은 라이브러리를 사용합니다. LlamaHub 덕분에 우리는 Unstructured와 직접 통합하여 모든 텍스트를 LlamaIndex가 수집할 수 있는 문서 형식으로 변환할 수 있습니다.

In [ ]:
!pip install llama-hub unstructured

### 그런 다음 UnstructuredReader를 사용하여 HTML 파일을 Document 개체 목록으로 구문 분석할 수 있습니다.

In [ ]:
from llama_hub.file.unstructured.base import UnstructuredReader
from pathlib import Path

years = [2022, 2021, 2020, 2019]

loader = UnstructuredReader()
doc_set = {}
all_docs = []
for year in years:
    year_docs = loader.load_data(
        file=Path(f"./data/UBER/UBER_{year}.html"), split_documents=False
    )
    # insert year metadata into each year
    for d in year_docs:
        d.metadata = {"year": year}
    doc_set[year] = year_docs
    all_docs.extend(year_docs)

### 각 연도별 벡터 지수 설정

### 먼저 각 연도에 대한 벡터 인덱스를 설정합니다. 각 벡터 인덱스를 통해 특정 연도의 10-K 제출에 대해 질문할 수 있습니다.

### 각 인덱스를 작성하여 디스크에 저장합니다.

In [ ]:
# initialize simple vector indices
from llama_index import VectorStoreIndex, ServiceContext, StorageContext

index_set = {}
service_context = ServiceContext.from_defaults(chunk_size=512)
for year in years:
    storage_context = StorageContext.from_defaults()
    cur_index = VectorStoreIndex.from_documents(
        doc_set[year],
        service_context=service_context,
        storage_context=storage_context,
    )
    index_set[year] = cur_index
    storage_context.persist(persist_dir=f"./storage/{year}")

### 디스크에서 인덱스를 로드하려면 다음을 수행하세요.

In [ ]:
# Load indices from disk
from llama_index import load_index_from_storage, StorageContext

index_set = {}
for year in years:
    storage_context = StorageContext.from_defaults(
        persist_dir=f"./storage/{year}"
    )
    cur_index = load_index_from_storage(
        storage_context, service_context=service_context
    )
    index_set[year] = cur_index

### 10-K 파일링 전체에 걸쳐 답변을 종합하기 위한 하위 질문 쿼리 엔진 설정

우리는 4년 간의 문서에 접근할 수 있으므로 해당 연도의 10-K 문서에 관한 질문뿐만 아니라 모든 10-K 서류에 대한 분석이 필요한 질문도 할 수 있습니다.

이 문제를 해결하기 위해 하위 질문 쿼리 엔진을 사용할 수 있습니다. 쿼리를 개별 벡터 인덱스로 응답하는 하위 쿼리로 분해하고 결과를 종합하여 전체 쿼리에 응답합니다.

In [ ]:
from llama_index.tools import QueryEngineTool, ToolMetadata

individual_query_engine_tools = [
    QueryEngineTool(
        query_engine=index_set[year].as_query_engine(),
        metadata=ToolMetadata(
            name=f"vector_index_{year}",
            description=f"useful for when you want to answer queries about the {year} SEC 10-K for Uber",
        ),
    )
    for year in years
]

In [ ]:
from llama_index.query_engine import SubQuestionQueryEngine

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=individual_query_engine_tools,
    service_context=service_context,
)

### 챗봇 에이전트 설정

우리는 LlamaIndex 데이터 에이전트를 사용하여 도구 세트에 액세스할 수 있는 외부 챗봇 에이전트를 설정합니다. 구체적으로 OpenAI API 함수 호출을 활용하는 OpenAIAgent를 사용하겠습니다. 우리는 위에서 정의한 하위 질문 쿼리 엔진을 위한 도구뿐만 아니라 각 지수(특정 연도에 해당)에 대해 이전에 정의한 별도의 도구를 사용하려고 합니다.

먼저 하위 질문 쿼리 엔진에 대한 QueryEngineTool을 정의합니다.

In [ ]:
query_engine_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="sub_question_query_engine",
        description="useful for when you want to answer queries that require analyzing multiple SEC 10-K documents for Uber",
    ),
)

그런 다음 위에서 정의한 도구를 에이전트용 단일 도구 목록으로 결합합니다.

In [ ]:
tools = individual_query_engine_tools + [query_engine_tool]

마지막으로 위에서 정의한 도구 목록을 전달하여 OpenAIAgent.from_tools를 호출하여 에이전트를 생성합니다.

In [ ]:
from llama_index.agent import OpenAIAgent

agent = OpenAIAgent.from_tools(tools, verbose=True)

### 에이전트 테스트

이제 다양한 쿼리로 에이전트를 테스트할 수 있습니다.

간단한 "hello" 쿼리로 테스트하면 에이전트는 어떤 도구도 사용하지 않습니다.

In [ ]:
response = agent.chat("hi, i am Joon")
print(str(response))

특정 연도의 10-k에 관한 쿼리로 테스트하면 에이전트는 관련 벡터 인덱스 도구를 사용합니다.

In [ ]:
response = agent.chat(
    "What were some of the biggest risk factors in 2020 for Uber?"
)
print(str(response))

마지막으로 수년간 위험 요소를 비교/대조하는 쿼리로 테스트하면 에이전트는 하위 질문 쿼리 엔진 도구를 사용합니다.

In [ ]:
cross_query_str = "Compare/contrast the risk factors described in the Uber 10-K across years. Give answer in bullet points."

response = agent.chat(cross_query_str)
print(str(response))

### 챗봇 루프 설정

In [ ]:
agent = OpenAIAgent.from_tools(tools)  # verbose=False by default

while True:
    text_input = input("User: ")
    if text_input == "exit":
        break
    response = agent.chat(text_input)
    print(f"Agent: {response}")